In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('dataset/train_split_partially_preprocessed.csv')

In [3]:
test_df = pd.read_csv('dataset/test_split_partially_preprocessed.csv')

In [6]:
assert(len(train_df.columns) == len(test_df.columns))

In [4]:
full_df = pd.concat([train_df, test_df])

In [7]:
full_df.columns

Index(['SERIAL', 'PERNUM', 'HHWT', 'CLUSTER', 'STRATA', 'PERWT', 'AGE',
       'YRMARR', 'YRNATUR', 'RACNUM', 'DEGFIELD', 'DEGFIELDD', 'OCC2010',
       'WKSWORK1', 'UHRSWORK', 'INCWAGE_CPIU_2010', 'PWSTATE2', 'PWCOUNTY',
       'PWTYPE', 'TRANTIME', 'isFemale', 'isAmericanIndian', 'isAsian',
       'isBlack', 'isPacificIslander', 'isWhite', 'isOtherRace',
       'hasPrivateHealthInsurance', 'hasEmployerHealthInsurance',
       'hasPurchasedPrivHealthInsurance', 'hasMilitaryHealthInsurance',
       'hasPublicHealthInsurance', 'hasMedicare', 'hasMedicaid',
       'hasVeteransHealthInsurance', 'hasIndianHealthInsurance',
       'hasHealthInsurance', 'isInSchool', 'carpools', 'isHispanic',
       'sameSexMarriage', 'mixedRaceMarriage', 'isGroupQuarters', 'bornInUS',
       'isMarried', 'wasMarried', 'neverMarried', 'speaksEnglish',
       'speaksOnlyEnglish', 'speaksEnglishWell', 'noSchooling', 'maxGrade4',
       'maxGrade8', 'maxSomeHS', 'highSchoolDiploma', 'someCollege',
       'assoc

In [17]:
full_df.dtypes.value_counts()

bool       47
int64      12
float64     8
object      5
dtype: int64

In [13]:
full_df.INCWAGE_CPIU_2010.mean()

45695.91944415493

In [55]:
def printColsValues(df, dfName, cols):
    print(f"Values for {dfName} ({len(df)} entries)\n")
    for col in cols:
        print(f'Information for column {col}:')
        if(df[col].dtype == 'float64' or df[col].dtype == 'int64'):
            print(df[col].describe())
        elif(df[col].dtype == 'object'):
            valueCounts = df[col].value_counts()
            print(f'True: {valueCounts[1]}, False: {valueCounts[0]}, Null: {df[col].isnull().values.sum()}')
        elif(df[col].dtype == 'bool'):
            valueCounts = df[col].value_counts()
            print(f'True: {valueCounts[1]}, False: {valueCounts[0]}')
        else:
            raise Exception(f'Unknown type {df[col].dtype}')
        print()

In [48]:
train_df.columns

Index(['SERIAL', 'PERNUM', 'HHWT', 'CLUSTER', 'STRATA', 'PERWT', 'AGE',
       'YRMARR', 'YRNATUR', 'RACNUM', 'DEGFIELD', 'DEGFIELDD', 'OCC2010',
       'WKSWORK1', 'UHRSWORK', 'INCWAGE_CPIU_2010', 'PWSTATE2', 'PWCOUNTY',
       'PWTYPE', 'TRANTIME', 'isFemale', 'isAmericanIndian', 'isAsian',
       'isBlack', 'isPacificIslander', 'isWhite', 'isOtherRace',
       'hasPrivateHealthInsurance', 'hasEmployerHealthInsurance',
       'hasPurchasedPrivHealthInsurance', 'hasMilitaryHealthInsurance',
       'hasPublicHealthInsurance', 'hasMedicare', 'hasMedicaid',
       'hasVeteransHealthInsurance', 'hasIndianHealthInsurance',
       'hasHealthInsurance', 'isInSchool', 'carpools', 'isHispanic',
       'sameSexMarriage', 'mixedRaceMarriage', 'isGroupQuarters', 'bornInUS',
       'isMarried', 'wasMarried', 'neverMarried', 'speaksEnglish',
       'speaksOnlyEnglish', 'speaksEnglishWell', 'noSchooling', 'maxGrade4',
       'maxGrade8', 'maxSomeHS', 'highSchoolDiploma', 'someCollege',
       'assoc

In [56]:
cols = [
    'AGE', 'INCWAGE_CPIU_2010', 'isFemale', 
    'isAmericanIndian', 'isAsian', 'isBlack', 'isPacificIslander', 'isWhite', 'isOtherRace',
    'hasHealthInsurance', 'hasPrivateHealthInsurance', 'hasPublicHealthInsurance',
    'isInSchool', 'isHispanic',
    'bornInUS', 'speaksEnglish',
    'isSelfEmployed', 'isPrivateSector', 'isPublicSector',
]

In [57]:
printColsValues(full_df, 'Full DF', cols)

Values for Full DF (1448821 entries)

Information for column AGE:
count    1.448821e+06
mean     4.365763e+01
std      1.499637e+01
min      1.600000e+01
25%      3.100000e+01
50%      4.300000e+01
75%      5.600000e+01
max      9.500000e+01
Name: AGE, dtype: float64

Information for column INCWAGE_CPIU_2010:
count    1.448821e+06
mean     4.569592e+04
std      5.690280e+04
min      0.000000e+00
25%      1.501700e+04
50%      3.128500e+04
75%      5.709500e+04
max      6.155270e+05
Name: INCWAGE_CPIU_2010, dtype: float64

Information for column isFemale:
True: 689933, False: 758888

Information for column isAmericanIndian:
True: 42172, False: 1406649

Information for column isAsian:
True: 114530, False: 1334291

Information for column isBlack:
True: 136251, False: 1312570

Information for column isPacificIslander:
True: 6072, False: 1442749

Information for column isWhite:
True: 330077, False: 1118744

Information for column isOtherRace:
True: 190660, False: 1258161

Information for co

In [58]:
printColsValues(train_df, 'Train split', cols)

Values for Train split (1231497 entries)

Information for column AGE:
count    1.231497e+06
mean     4.365052e+01
std      1.498938e+01
min      1.600000e+01
25%      3.100000e+01
50%      4.300000e+01
75%      5.600000e+01
max      9.500000e+01
Name: AGE, dtype: float64

Information for column INCWAGE_CPIU_2010:
count    1.231497e+06
mean     4.569919e+04
std      5.685871e+04
min      0.000000e+00
25%      1.501700e+04
50%      3.128500e+04
75%      5.709500e+04
max      6.155270e+05
Name: INCWAGE_CPIU_2010, dtype: float64

Information for column isFemale:
True: 586371, False: 645126

Information for column isAmericanIndian:
True: 35774, False: 1195723

Information for column isAsian:
True: 97376, False: 1134121

Information for column isBlack:
True: 115975, False: 1115522

Information for column isPacificIslander:
True: 5145, False: 1226352

Information for column isWhite:
True: 280628, False: 950869

Information for column isOtherRace:
True: 162150, False: 1069347

Information for 

In [59]:
printColsValues(test_df, 'Test split', cols)

Values for Test split (217324 entries)

Information for column AGE:
count    217324.000000
mean         43.697893
std          15.035873
min          16.000000
25%          31.000000
50%          44.000000
75%          56.000000
max          95.000000
Name: AGE, dtype: float64

Information for column INCWAGE_CPIU_2010:
count    217324.000000
mean      45677.388664
std       57152.155656
min           0.000000
25%       14860.000000
50%       31285.000000
75%       57095.000000
max      615527.000000
Name: INCWAGE_CPIU_2010, dtype: float64

Information for column isFemale:
True: 103562, False: 113762

Information for column isAmericanIndian:
True: 6398, False: 210926

Information for column isAsian:
True: 17154, False: 200170

Information for column isBlack:
True: 20276, False: 197048

Information for column isPacificIslander:
True: 927, False: 216397

Information for column isWhite:
True: 49449, False: 167875

Information for column isOtherRace:
True: 28510, False: 188814

Information 

In [62]:
df_0to25 = full_df[full_df['INCWAGE_CPIU_2010'] <= 25000]
printColsValues(df_0to25, 'Under 25k salary', cols)

Values for Under 25k salary (575690 entries)

Information for column AGE:
count    575690.000000
mean         40.838785
std          17.391372
min          16.000000
25%          25.000000
50%          39.000000
75%          56.000000
max          95.000000
Name: AGE, dtype: float64

Information for column INCWAGE_CPIU_2010:
count    575690.000000
mean      10871.010057
std        8153.123469
min           0.000000
25%        3128.000000
50%       10402.000000
75%       18771.000000
max       24950.000000
Name: INCWAGE_CPIU_2010, dtype: float64

Information for column isFemale:
True: 261056, False: 314634

Information for column isAmericanIndian:
True: 19837, False: 555853

Information for column isAsian:
True: 40676, False: 535014

Information for column isBlack:
True: 65353, False: 510337

Information for column isPacificIslander:
True: 2679, False: 573011

Information for column isWhite:
True: 149924, False: 425766

Information for column isOtherRace:
True: 95530, False: 480160

Inf

In [63]:
df_25to50 = full_df[(full_df['INCWAGE_CPIU_2010'] > 25000) & (full_df['INCWAGE_CPIU_2010'] <= 50000)]
printColsValues(df_25to50, '25k to 50k salary', cols)

Values for 25k to 50k salary (428639 entries)

Information for column AGE:
count    428639.000000
mean         43.899466
std          13.571027
min          16.000000
25%          32.000000
50%          43.000000
75%          55.000000
max          95.000000
Name: AGE, dtype: float64

Information for column INCWAGE_CPIU_2010:
count    428639.000000
mean      36150.342171
std        6974.695765
min       25028.000000
25%       30503.000000
50%       35195.000000
75%       41452.000000
max       49273.000000
Name: INCWAGE_CPIU_2010, dtype: float64

Information for column isFemale:
True: 208988, False: 219651

Information for column isAmericanIndian:
True: 13246, False: 415393

Information for column isAsian:
True: 26492, False: 402147

Information for column isBlack:
True: 42469, False: 386170

Information for column isPacificIslander:
True: 1910, False: 426729

Information for column isWhite:
True: 95268, False: 333371

Information for column isOtherRace:
True: 57936, False: 370703

Inf

In [64]:
df_50to75 = full_df[(full_df['INCWAGE_CPIU_2010'] > 50000) & (full_df['INCWAGE_CPIU_2010'] <= 75000)]
printColsValues(df_50to75, '50k to 75k salary', cols)

Values for 50k to 75k salary (215793 entries)

Information for column AGE:
count    215793.000000
mean         45.807992
std          12.296983
min          16.000000
25%          36.000000
50%          46.000000
75%          56.000000
max          95.000000
Name: AGE, dtype: float64

Information for column INCWAGE_CPIU_2010:
count    215793.000000
mean      60421.608931
std        6996.685487
min       50056.000000
25%       54748.000000
50%       58659.000000
75%       66480.000000
max       74301.000000
Name: INCWAGE_CPIU_2010, dtype: float64

Information for column isFemale:
True: 91910, False: 123883

Information for column isAmericanIndian:
True: 5217, False: 210576

Information for column isAsian:
True: 17541, False: 198252

Information for column isBlack:
True: 16482, False: 199311

Information for column isPacificIslander:
True: 808, False: 214985

Information for column isWhite:
True: 40640, False: 175153

Information for column isOtherRace:
True: 20762, False: 195031

Inform

In [65]:
df_over75 = full_df[(full_df['INCWAGE_CPIU_2010'] > 75000)]
printColsValues(df_over75, 'Over 75k salary', cols)

Values for Over 75k salary (228699 entries)

Information for column AGE:
count    228699.000000
mean         48.271046
std          11.389016
min          16.000000
25%          39.000000
50%          48.000000
75%          57.000000
max          95.000000
Name: AGE, dtype: float64

Information for column INCWAGE_CPIU_2010:
count    228699.000000
mean     137354.641476
std       91532.722240
min       75083.000000
25%       86033.000000
50%      102457.000000
75%      141563.000000
max      615527.000000
Name: INCWAGE_CPIU_2010, dtype: float64

Information for column isFemale:
True: 74401, False: 154298

Information for column isAmericanIndian:
True: 3872, False: 224827

Information for column isAsian:
True: 29821, False: 198878

Information for column isBlack:
True: 11947, False: 216752

Information for column isPacificIslander:
True: 675, False: 228024

Information for column isWhite:
True: 44245, False: 184454

Information for column isOtherRace:
True: 16432, False: 212267

Informat

In [71]:
for col in cols:
    if full_df[col].dtype != 'bool':
        continue

    assert(not full_df[col].isna().values.any())

    print(f'Of the people for whom {col} is true, salary information is:')
    print(full_df[full_df[col]].INCWAGE_CPIU_2010.describe())
    print(f'Of the people for whom {col} is false, salary information is:')
    print(full_df[~full_df[col]].INCWAGE_CPIU_2010.describe())
    print("\n")

Of the people for whom isFemale is true, salary information is:
count    689933.000000
mean      37428.078499
std       44020.824309
min           0.000000
25%       11732.000000
50%       27374.000000
75%       47709.000000
max      615527.000000
Name: INCWAGE_CPIU_2010, dtype: float64
Of the people for whom isFemale is false, salary information is:
count    758888.000000
mean      53212.517818
std       65583.938792
min           0.000000
25%       17989.000000
50%       37542.000000
75%       64916.000000
max      615527.000000
Name: INCWAGE_CPIU_2010, dtype: float64


Of the people for whom isAmericanIndian is true, salary information is:
count     42172.000000
mean      35160.549108
std       40239.010535
min           0.000000
25%       11732.000000
50%       26748.000000
75%       46927.000000
max      533404.000000
Name: INCWAGE_CPIU_2010, dtype: float64
Of the people for whom isAmericanIndian is false, salary information is:
count    1.406649e+06
mean     4.601177e+04
std     